The train folder contains 25,000 images of dogs and cats. Each image in this folder has the label as part of the filename. The test folder contains 12,500 images, named according to a numeric id. For each image in the test set, you should predict a probability that the image is a dog (1 = dog, 0 = cat).

In [1]:
import os  # 处理字符串路径
import glob  # 查找文件
from keras.models import Sequential  # 导入Sequential模型
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import numpy as np
from pandas import Series, DataFrame
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from six.moves import range

Using TensorFlow backend.


In [2]:
#加载数据
import os
from PIL import Image
import numpy as np
#读取文件夹train下的25000张图片，图片为彩色图，所以为3通道，
#如果是将彩色图作为输入,图像大小224*224
DIR = "/Users/chris/gitspace/data"
def load_data():
    sed = 1000
    data = np.empty((20000,224,224,3),dtype="float32")
    label = np.empty((20000,))
    imgs = os.listdir(DIR + "/cat_dog/train/")
    num = len(imgs)
    times = 0
    time = 0
    for i in range(num):
        if imgs[i].split('.')[0] == 'cat':
            if times ==1000:
                continue
            img = Image.open(DIR + "/cat_dog/train/" + imgs[i])

            arr = np.asarray(img, dtype="float32")
            arr.resize((224,224,3))
            data[i, :, :, :] = arr
            label[i] = 0
            times +=1
        else:
            img = Image.open(DIR + "/cat_dog/train/" + imgs[i])
            arr = np.asarray(img, dtype="float32")
            arr.resize((224, 224, 3))
            data[1000 + time, :, :, :] = arr
            label[1000 + time] = 1
            time +=1
            if time == 1000:
                break

    return data,label

data,label = load_data()
print(data.shape)
train_data = data[:1800]
train_labels = label[:1800]
validation_data = data[1800:]
validation_labels = label[1800:]

(20000, 224, 224, 3)


In [3]:
model = Sequential()
#第一个卷积层，4个卷积核，每个卷积核大小5*5。
#激活函数用tanh
#你还可以在model.add(Activation('tanh'))后加上dropout的技巧: model.add(Dropout(0.5))
model.add(Convolution2D(4, 5, 5,input_shape=(224, 224,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#第二个卷积层，8个卷积核，每个卷积核大小3*3。
#激活函数用tanh
#采用maxpooling，poolsize为(2,2)
model.add(Convolution2D(8, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#第三个卷积层，16个卷积核，每个卷积核大小3*3
#激活函数用tanh
#采用maxpooling，poolsize为(2,2)
model.add(Convolution2D(16, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#全连接层，先将前一层输出的二维特征图flatten为一维的。

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#sigmoid分类，输出是2类别
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.fit(train_data, train_labels,
         nb_epoch=10, batch_size=100,
         validation_data=(validation_data, validation_labels))


ValueError: Only call `sigmoid_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)

In [ ]:
predictions = model.predict(validation_data, verbose=0)